<a href="https://colab.research.google.com/github/Leucocitokiller/Proyecto-Fina-NLP/blob/main/Proyecto_final_NLP_Redes_Neuronales_Libenson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importación de Librerías.

In [1]:
import urllib.request
import numpy as np
import pandas as pd
import os
import time
import sys
#-----librerias para trabajar PLN
!python -m spacy download es_core_news_md
import spacy
import es_core_news_md
#es_core_news_md Medium (modelo mediano):
#Es más pesado y más lento que el sm, pero mucho más preciso. Tiene vectores de palabras, entiende mejor el significado de las palabras.

#-----instalación d librerias para análisis de sentimientos.
!pip install spacy spacy-transformers
!pip install pysentimiento
from pysentimiento import create_analyzer

#----librerias para normalización de textos
import re
from unicodedata import normalize
import unicodedata
from collections import Counter


#----librerias para graficar y wordcloud.
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

#----librerías para trabajar con TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
#----libreria para trabajar con BoW.
from sklearn.feature_extraction.text import CountVectorizer
#----librerias para Machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 17.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

# Procesamiento de la Fuente de Datos.

## Conexión con la fuente de datos.


In [12]:
# Diccionario con las fuentes y sus URLs
filepath_dict = {
    'yelp': 'https://raw.githubusercontent.com/Leucocitokiller/Proyecto-Fina-NLP/main/yelp_comentarios.csv',
    'amazon': 'https://raw.githubusercontent.com/Leucocitokiller/Proyecto-Fina-NLP/main/amazon_cells_comentarios.csv'

}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['Comentario', 'Valor'], sep=';', encoding='latin-1')
    df['Origen'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
df.head(1100)

,Comentario,Valor,Origen
0,Wow ... Me encantó este lugar.,1,yelp
1,La corteza no es buena.,0,yelp
2,No sabroso y la textura era simplemente desagr...,0,yelp
3,Me detuve durante las vacaciones de fines de m...,1,yelp
4,"La selección en el menú era genial, al igual q...",1,yelp
...,...,...,...
96,"Si planeas usar esto en un auto, olvídalo.",0,amazon
97,Encontré este producto demasiado grande.,0,amazon
98,Lo mejor que he encontrado hasta ahora ... He ...,1,amazon
99,Estoy muy decepcionado con mi decisión.,0,amazon


## Normalización de la Fuente de datos.


### Eliminación de signos de puntuación.

In [13]:
# Definición de función para eliminar los signos de puntuación utilizando re, pero considerando no borrar las vocales con acento.

def remove_punctuation(text):
    # Normaliza el texto a NFKD para separar letras y sus tildes
    text = unicodedata.normalize('NFKD', text)
    # Elimina los caracteres diacríticos (como las tildes)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    # Elimina todo lo que no sea letras, números o espacios
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


# Aplicar la función a la columna 'review_lower'
df['Comentarios'] = df['Comentario'].apply(remove_punctuation)

In [14]:
df

,Comentario,Valor,Origen,Comentarios
0,Wow ... Me encantó este lugar.,1,yelp,Wow Me encanto este lugar
1,La corteza no es buena.,0,yelp,La corteza no es buena
2,No sabroso y la textura era simplemente desagr...,0,yelp,No sabroso y la textura era simplemente desagr...
3,Me detuve durante las vacaciones de fines de m...,1,yelp,Me detuve durante las vacaciones de fines de m...
4,"La selección en el menú era genial, al igual q...",1,yelp,La seleccion en el menu era genial al igual qu...
...,...,...,...,...
995,La pantalla se mancha fácilmente porque toca l...,0,amazon,La pantalla se mancha facilmente porque toca l...
996,Qué pedazo de chatarra ... Pierdo más llamadas...,0,amazon,Que pedazo de chatarra Pierdo mas llamadas en...
997,El artículo no coincide con la imagen.,0,amazon,El articulo no coincide con la imagen
998,Lo Ãºnico que me decepcionó es el puerto infra...,0,amazon,Lo Aonico que me decepciono es el puerto infra...


### Reducir a minúsculas el texto.

In [15]:
# Create a new column 'Comentarios_lower' with lowercase values from 'Comentario'
df['Comentarios_lower'] = df['Comentarios'].str.lower()

In [16]:
df

,Comentario,Valor,Origen,Comentarios,Comentarios_lower
0,Wow ... Me encantó este lugar.,1,yelp,Wow Me encanto este lugar,wow me encanto este lugar
1,La corteza no es buena.,0,yelp,La corteza no es buena,la corteza no es buena
2,No sabroso y la textura era simplemente desagr...,0,yelp,No sabroso y la textura era simplemente desagr...,no sabroso y la textura era simplemente desagr...
3,Me detuve durante las vacaciones de fines de m...,1,yelp,Me detuve durante las vacaciones de fines de m...,me detuve durante las vacaciones de fines de m...
4,"La selección en el menú era genial, al igual q...",1,yelp,La seleccion en el menu era genial al igual qu...,la seleccion en el menu era genial al igual qu...
...,...,...,...,...,...
995,La pantalla se mancha fácilmente porque toca l...,0,amazon,La pantalla se mancha facilmente porque toca l...,la pantalla se mancha facilmente porque toca l...
996,Qué pedazo de chatarra ... Pierdo más llamadas...,0,amazon,Que pedazo de chatarra Pierdo mas llamadas en...,que pedazo de chatarra pierdo mas llamadas en...
997,El artículo no coincide con la imagen.,0,amazon,El articulo no coincide con la imagen,el articulo no coincide con la imagen
998,Lo Ãºnico que me decepcionó es el puerto infra...,0,amazon,Lo Aonico que me decepciono es el puerto infra...,lo aonico que me decepciono es el puerto infra...


### Convertir a número la columna Valor para su postprocesamiento.

In [17]:
# Convertimos la columna rating a valor numérico
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')

In [18]:
df['Valor']

,Valor
0,1
1,0
2,0
3,1
4,1
...,...
995,0
996,0
997,0
998,0


# NLP

## Pre Procesamiento.

### Generación del objeto de SPacy para utilizar en el procesamiento del texto en español.

In [19]:
nlp = es_core_news_md.load()

### Convertir texto a minúsculas y Tokenización.

In [21]:
df['Comentarios_tokenizados'] = df['Comentarios_lower'].apply(lambda text: nlp(text))
df[['Comentarios_lower','Comentarios_tokenizados']].head()

,Comentarios_lower,Comentarios_tokenizados
0,wow me encanto este lugar,"(wow, , me, encanto, este, lugar)"
1,la corteza no es buena,"(la, corteza, no, es, buena)"
2,no sabroso y la textura era simplemente desagr...,"(no, sabroso, y, la, textura, era, simplemente..."
3,me detuve durante las vacaciones de fines de m...,"(me, detuve, durante, las, vacaciones, de, fin..."
4,la seleccion en el menu era genial al igual qu...,"(la, seleccion, en, el, menu, era, genial, al,..."


### Remoción de StopWords

In [24]:
# Lista de palabras que NO queremos eliminar (tienen carga emocional)
palabras_sentimiento = {
    # Positivas
    "bueno", "buena","si","buenísimo", "excelentes", "excelente", "genial", "maravilloso", "maravilla", "fantástico", "fabuloso", "increíble",
    "perfecto", "perfecta", "agradable", "satisfecho", "satisfecha", "contento", "contenta", "encantado", "encantada",
    "amable", "simpático", "simpática", "rápido", "rápida", "cómodo", "cómoda", "eficaz", "eficiente", "fácil",
    "recomendable", "ideal", "espectacular", "feliz", "brillante", "cumplió", "cumple", "funciona", "funciona bien",
    "inmejorable", "confiable", "duradero", "cumplidor", "seguro", "preciso", "elegante", "atento", "responsable",
    "acertado", "destacado", "excepcional", "impecable", "sensacional", "útil", "accesible", "económico", "funcional",
    "intuitivo", "conveniente", "hermoso", "linda", "precioso", "excelente calidad", "vale la pena",

    # Negativas
    "malo","no", "mala", "mal", "pésimo", "pésima","nunca", "horrible", "fatal", "insoportable", "lento", "lenta", "incómodo", "incómoda",
    "decepcionante", "decepcionado", "decepcionada", "sucio", "sucia", "caro", "cara", "inútil", "deficiente", "desagradable",
    "complicado", "problemático", "estafa", "engañado", "engañada", "roto", "rota", "desastroso", "error", "errores",
    "retraso", "tardanza", "frágil", "inestable", "poco fiable", "nunca más", "no volveré", "no recomiendo", "no sirve",
    "no funciona", "arruinado", "falló", "fallando", "demora", "pésima atención", "servicio malo", "mala calidad", "molesto",
    "defecto", "problemas", "fallas", "sin sentido", "basura", "pérdida de dinero", "decepción"
}

# Actualizamos spaCy para que NO considere esas palabras como stopwords
for palabra in palabras_sentimiento:
    lex = nlp.vocab[palabra]
    lex.is_stop = False



def parse_and_remove_stopwords(doc):
    """
    Remueve las stopwords de un objeto spaCy Doc.
    """
    # Filtrar stopwords y obtener los tokens como texto
    tokens_filtrados = [token.text for token in doc if not token.is_stop]
    return tokens_filtrados

# Aplicar la función al DataFrame
df['Comentarios_sin_StopWords'] = df['Comentarios_tokenizados'].apply(parse_and_remove_stopwords)

df[['Comentarios_tokenizados','Comentarios_sin_StopWords']].head()

,Comentarios_tokenizados,Comentarios_sin_StopWords
0,"(wow, , me, encanto, este, lugar)","[wow, , encanto, lugar]"
1,"(la, corteza, no, es, buena)","[corteza, no, buena]"
2,"(no, sabroso, y, la, textura, era, simplemente...","[no, sabroso, textura, simplemente, desagradable]"
3,"(me, detuve, durante, las, vacaciones, de, fin...","[detuve, vacaciones, fines, mayo, recomendacio..."
4,"(la, seleccion, en, el, menu, era, genial, al,...","[seleccion, menu, genial, precios]"


### Lematizado.

In [25]:
def lematizar_sin_stopwords(doc):
    """
    Devuelve una lista de lemas excluyendo las stopwords.

    Parámetro:
    - doc: objeto spaCy Doc

    Retorna:
    - Lista de lemas (str) sin stopwords
    """
    return [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

# Aplicar la función y guardar el resultado en una nueva columna
df['Comentarios_lema'] = df['Comentarios_tokenizados'].apply(lematizar_sin_stopwords)

df[['Comentarios_tokenizados','Comentarios_lema']].head(20)

,Comentarios_tokenizados,Comentarios_lema
0,"(wow, , me, encanto, este, lugar)","[wow, encantar, lugar]"
1,"(la, corteza, no, es, buena)","[corteza, no, buen]"
2,"(no, sabroso, y, la, textura, era, simplemente...","[no, sabroso, textura, simplemente, desagradable]"
3,"(me, detuve, durante, las, vacaciones, de, fin...","[detener, vacación, fines, mayo, recomendación..."
4,"(la, seleccion, en, el, menu, era, genial, al,...","[seleccion, menu, genial, precio]"
5,"(ahora, me, estoy, enojando, y, quiero, mi, ma...","[enojar, querer, maldito, pho]"
6,"(honestamente, no, sabia, tan, fresco)","[honestamente, no, sabio, fresco]"
7,"(las, papas, eran, como, caucho, y, se, notaba...","[papa, caucho, notar, habiar, preparar, tiempo..."
8,"(las, patatas, fritas, tambien, eran, geniales)","[patata, frita, genial]"
9,"(un, gran, toque)",[toque]


## Procesamiento

### Conteo de Palabras mas comunes.